# StyleGAN3 Training — Latent Resonance Spectrograms

Train StyleGAN3 on 512x512 grayscale spectrogram images for the **Latent Resonance: Facial Acoustics** project.

Uses the **stylegan3-t** (translation-equivariant) configuration, which is well-suited for spectrograms where the horizontal axis represents time.

**Requirements:** Run this notebook on Google Colab with a GPU runtime (T4 or better).

## 1. Setup & GPU Check

In [1]:
!nvidia-smi

# Reinstall PyTorch with CUDA support (pip install torch can pull CPU-only builds)
!pip install torch torchvision
!pip install ninja pillow requests

import torch
assert torch.cuda.is_available(), "CUDA not available — make sure you selected a GPU runtime (Runtime → Change runtime type → GPU)"
print(f"PyTorch {torch.__version__}, CUDA {torch.version.cuda}, GPU: {torch.cuda.get_device_name(0)}")

Sun Feb  1 22:16:24 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 2. Clone StyleGAN3

In [2]:
import os
import sys
import shutil

# Remove previous clone so patches apply cleanly
if os.path.exists("stylegan3"):
    shutil.rmtree("stylegan3")

!git clone https://github.com/NVlabs/stylegan3.git

sys.path.insert(0, "stylegan3")

Cloning into 'stylegan3'...
remote: Enumerating objects: 212, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 212 (delta 99), reused 90 (delta 90), pack-reused 49 (from 1)
Receiving objects: 100% (212/212), 4.16 MiB | 9.45 MiB/s, done.
Resolving deltas: 100% (108/108), done.


In [3]:
import os
import pathlib
import shutil
import subprocess

# ── Patch 1: Fix InfiniteSampler for PyTorch ≥2.4 ────────────────────────────
misc_path = pathlib.Path("stylegan3/torch_utils/misc.py")
src = misc_path.read_text()
src = src.replace("super().__init__(dataset)", "super().__init__()")
misc_path.write_text(src)
print(f"Patched {misc_path}: InfiniteSampler fix")

# ── Patch 2: Fix Adam betas int → float for PyTorch ≥2.9 ────────────────────
train_path = pathlib.Path("stylegan3/train.py")
src = train_path.read_text()
src = src.replace("betas=[0,0.99]", "betas=[0.0,0.99]")
train_path.write_text(src)
print(f"Patched {train_path}: Adam betas fix")

# ── Patch 3: Try CUDA ops compilation, fall back if it fails ─────────────────
# Set env vars for CUDA compilation (T4 = compute capability 7.5)
os.environ["TORCH_CUDA_ARCH_LIST"] = "7.5"
os.environ["TORCH_EXTENSIONS_DIR"] = "/tmp/torch_extensions"

# Clean stale builds
if os.path.exists("/tmp/torch_extensions"):
    shutil.rmtree("/tmp/torch_extensions")

# Test compilation in a subprocess (same env as training will use)
result = subprocess.run(
    ["python", "-c",
     "import sys; sys.path.insert(0,'stylegan3'); "
     "from torch_utils.ops import bias_act; "
     "assert bias_act._init(), 'init failed'"],
    capture_output=True, text=True, timeout=120,
)

CUDA_OPS_OK = result.returncode == 0
if CUDA_OPS_OK:
    print("Custom CUDA ops compiled successfully — using fused kernels")
else:
    print("CUDA ops compilation failed, falling back to native PyTorch + reduced model")
    print(f"  stderr (last 300 chars): ...{result.stderr[-300:]}")
    # Disable custom ops so ref implementations are used
    ops_dir = pathlib.Path("stylegan3/torch_utils/ops")
    for name in ["bias_act.py", "upfirdn2d.py", "filtered_lrelu.py"]:
        p = ops_dir / name
        src = p.read_text()
        src = src.replace("def _init():", "def _init():\n    return False")
        p.write_text(src)
        print(f"  Patched {p}")
    print("  Will use --cbase=16384 --cmax=256 to fit in T4 VRAM")

Patched stylegan3/torch_utils/misc.py: InfiniteSampler fix
Patched stylegan3/train.py: Adam betas fix
CUDA ops compilation failed, falling back to native PyTorch + reduced model
  stderr (last 300 chars): ...^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 1387, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1360, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1324, in _find_and_load_unlocked
ModuleNotFoundError: No module named 'bias_act_plugin'

  Patched stylegan3/torch_utils/ops/bias_act.py
  Patched stylegan3/torch_utils/ops/upfirdn2d.py
  Patched stylegan3/torch_utils/ops/filtered_lrelu.py
  Will use --cbase=16384 --cmax=256 to fit in T4 VRAM


## 3. Upload Dataset

Get your `data/spectrograms/*.png` files into the Colab runtime. Pick **one** of the options below:

- **Option A** — Download from a shared Google Drive folder/file using `gdown`
- **Option B** — Download from any public URL using `wget`
- **Option C** — Drag and drop files into the Colab file browser sidebar (📁 icon on the left)

In [4]:
import os
import glob
import shutil

# ── Option A: Download a shared Google Drive folder ──────────────────────────
# 1. Share your Drive folder (Anyone with the link → Viewer)
# 2. Copy link → extract the ID after /folders/
# Uncomment the two lines below and paste your folder ID:
#
# !pip install -q gdown
# !gdown --folder "https://drive.google.com/drive/folders/YOUR_FOLDER_ID" -O ./spectrograms --remaining-ok

# ── Option B: Download a zip from any URL ────────────────────────────────────
# Host your spectrograms.zip somewhere (Drive shared link, S3, etc.)
# For a Google Drive *file* link, use gdown:
#
!pip install -q gdown
!gdown --fuzzy "https://drive.google.com/file/d/1x9ABdgP06h00cLygu2qSOXCSCma1sKFA/view?usp=sharing" -O spectrograms.zip
!unzip -qo spectrograms.zip -d ./spectrograms_raw
pngs = glob.glob("spectrograms_raw/**/*.png", recursive=True)
os.makedirs("./spectrograms", exist_ok=True)
for p in pngs:
    shutil.copy(p, "./spectrograms/")

# ── Option C: Drag & drop ────────────────────────────────────────────────────
# 1. Click the 📁 icon in the left sidebar to open the file browser
# 2. Create a "spectrograms" folder
# 3. Drag your PNG files (or a zip) into it
# If you uploaded a zip, uncomment:
# !unzip -qo spectrograms.zip -d ./spectrograms

# ── Set the final path (should contain *.png files) ──────────────────────────
DATASET_PATH = "./spectrograms"  # <-- adjust if your images end up elsewhere

# Verify
pngs = [f for f in os.listdir(DATASET_PATH) if f.endswith(".png")]
print(f"Found {len(pngs)} PNG files in {DATASET_PATH}")

Downloading...
From (original): https://drive.google.com/uc?id=1x9ABdgP06h00cLygu2qSOXCSCma1sKFA
From (redirected): https://drive.google.com/uc?id=1x9ABdgP06h00cLygu2qSOXCSCma1sKFA&confirm=t&uuid=7bfff5b4-b4e6-4d55-9bf2-9f27be8da6ea
To: /content/spectrograms.zip
100% 57.8M/57.8M [00:00<00:00, 115MB/s] 
Found 435 PNG files in ./spectrograms


## 4. Prepare Dataset

StyleGAN3 requires images packaged as a zip archive. The `dataset_tool.py` script handles grayscale detection and resolution validation automatically.

## 5. Configure Training

Parameters auto-adjust based on whether CUDA ops compiled:

| Scenario | `BATCH_SIZE` | `CBASE` | `CMAX` |
|----------|-------------|---------|--------|
| CUDA ops OK | 8 | 32768 | 512 |
| Native fallback | 4 | 16384 | 256 |

Reduced `cbase`/`cmax` halve the channel counts, which is needed because the native PyTorch reference ops use significantly more VRAM than the fused CUDA kernels. For a 435-image dataset this capacity is sufficient.

In [13]:
import os
os.environ["PYTORCH_ALLOC_CONF"] = "expandable_segments:True"

GPUS = 1
GAMMA = 2.0
SNAP = 10
KIMG = 1000
CFG = "stylegan3-t"
MIRROR = False
METRICS = "none"
RESUME = ""

# Adjust capacity based on whether CUDA ops compiled
if CUDA_OPS_OK:
    BATCH_SIZE = 8
    CBASE = 32768
    CMAX = 512
    print("Using full model capacity with fused CUDA kernels")
else:
    BATCH_SIZE = 2
    CBASE = 8192    # quarter — ref ops pad/conv are extremely memory-hungry
    CMAX = 128
    print(f"Using reduced model (cbase={CBASE}, cmax={CMAX}, batch={BATCH_SIZE}) with native PyTorch ops")

Using reduced model (cbase=8192, cmax=128, batch=2) with native PyTorch ops


## 6. Train

In [ ]:
import torch
torch.cuda.empty_cache()

resume_flag = f"--resume={RESUME}" if RESUME else ""
mirror_int = 1 if MIRROR else 0

!python stylegan3/train.py \
    --outdir=./training-runs \
    --cfg={CFG} \
    --data=./spectrograms.zip \
    --gpus={GPUS} \
    --batch={BATCH_SIZE} \
    --batch-gpu={BATCH_SIZE} \
    --gamma={GAMMA} \
    --snap={SNAP} \
    --kimg={KIMG} \
    --mirror={mirror_int} \
    --metrics={METRICS} \
    --cbase={CBASE} \
    --cmax={CMAX} \
    --mbstd-group={BATCH_SIZE} \
    {resume_flag}


Training options:
{
  "G_kwargs": {
    "class_name": "training.networks_stylegan3.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "channel_base": 8192,
    "channel_max": 128,
    "magnitude_ema_beta": 0.9999306876841536
  },
  "D_kwargs": {
    "class_name": "training.networks_stylegan2.Discriminator",
    "block_kwargs": {
      "freeze_layers": 0
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 2
    },
    "channel_base": 8192,
    "channel_max": 128
  },
  "G_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0.0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.0025
  },
  "D_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0.0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.002
  },
  "loss_kwargs": {
    "class_name": "training.loss.StyleGAN2Loss",
    "r1_gamma": 2.0
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "prefetch_fa

## 7. Generate Samples

Load the latest checkpoint and generate sample spectrograms to verify training quality.

In [ ]:
import glob
import pickle

import matplotlib.pyplot as plt
import torch

# Find the latest snapshot pickle
pkls = sorted(glob.glob("training-runs/**/*.pkl", recursive=True))
assert pkls, "No snapshots found — has training completed at least one snapshot?"
latest_pkl = pkls[-1]
print(f"Loading {latest_pkl}")

with open(latest_pkl, "rb") as f:
    G = pickle.load(f)["G_ema"].cuda().eval()

# Generate 4 samples
z = torch.randn(4, G.z_dim, device="cuda")
with torch.no_grad():
    imgs = G(z, None)

# Display as a grid
fig, axes = plt.subplots(1, 4, figsize=(16, 4))
for i, ax in enumerate(axes):
    img = imgs[i, 0].cpu().numpy()  # single channel
    ax.imshow(img, cmap="magma", aspect="auto")
    ax.set_title(f"Sample {i}")
    ax.axis("off")
plt.suptitle("Generated Spectrograms")
plt.tight_layout()
plt.show()

## 8. Save Results

Zip the training results. Download via the Colab file browser (📁 sidebar → right-click → Download), or upload to Drive with `gdown`.

In [ ]:
import shutil

# Create a zip of the training runs
shutil.make_archive("training-runs", "zip", ".", "training-runs")
print("Created training-runs.zip")
print("Download it from the file browser: click 📁 in the left sidebar → right-click training-runs.zip → Download")